<a href="https://colab.research.google.com/github/feijaum/aluraproj/blob/main/ProjetoAlura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Backend**

In [30]:
# Importando as bibliotecas necessárias
import google.generativeai as genai
import google.colab.userdata as userdata
from IPython.display import Markdown, display

# Configurando a API Key do Google AI Studio
# A chave da API está armazenada como um Secret no Colab com o nome padrão
API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=API_KEY)

# Define a versão do modelo Gemini a ser utilizada
GEMINI_MODEL = 'gemini-2.5-flash-preview-04-17'

def exibir_markdown(texto):
    """Exibe texto formatado em Markdown."""
    display(Markdown(texto))
    return texto # Retorna o texto para ser exibido na Textarea

# --- Classes com integração básica com Gemini ---

class AgentePesquisador:
    def processar(self, tema):
        """Utiliza o Gemini para gerar um resumo em tópicos sobre o tema."""
        print(f"\nChamando Agente Pesquisador para o tema: {tema}")
        try:
            model = genai.GenerativeModel(GEMINI_MODEL)
            prompt = f"Gere um resumo em tópicos, conciso e dinâmico, sobre o seguinte tema: {tema}. Adapte a linguagem para diferentes níveis de conhecimento."
            response = model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Ocorreu um erro ao processar a pesquisa: {e}"

class AgenteQuestionador:
    def processar(self, tema):
        """Utiliza o Gemini para gerar um questionário com gabarito sobre o tema."""
        print(f"\nChamando Agente Questionador para o tema: {tema}")
        try:
            model = genai.GenerativeModel(GEMINI_MODEL)
            prompt = f"""Gere um questionário de múltipla escolha com 10 questões específicas sobre "{tema}".
            Inclua o gabarito ao final do questionário.
            Caso o assunto seja muito amplo, foque em um tópico específico dentro dele e indique o tema do questionário no título.
            Adicione uma mensagem ao usuário incentivando a especificidade na busca para questionários mais direcionados.
            Formate o questionário de forma clara, com as questões numeradas e as opções de resposta (a, b, c, d, e)."""
            response = model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Ocorreu um erro ao processar o questionário: {e}"

class AgentePlanejador:
    def processar(self, tema):
        """Utiliza o Gemini para gerar um cronograma de estudos sobre o tema."""
        print(f"\nChamando Agente Planejador para o tema: {tema}")
        try:
            model = genai.GenerativeModel(GEMINI_MODEL)
            prompt = f"""Elabore um cronograma de estudos de 30 dias para aprender sobre "{tema}" do zero.
            Considere uma disponibilidade de 2 horas de estudo por dia.
            O cronograma deve indicar os temas principais, a ordem de estudo e a alocação de dias para cada tópico."""
            response = model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Ocorreu um erro ao processar o cronograma: {e}"

class AgenteProfessor:
    def compilar_info(self, pesquisa, questionario, cronograma):
        """Compila as informações dos outros agentes em um formato de documento."""
        print("\nChamando Agente Professor para compilar informações.")
        documento = f"""
# Documento de Estudo Completo

## Pesquisa

{pesquisa}

---

## Quiz

{questionario}

---

## Planejamento de Estudos

{cronograma}

---

*Este documento foi gerado por um assistente de estudos com múltiplos agentes utilizando o Google AI Studio.*
*Em um ambiente real, este conteúdo poderia ser exportado para o Google Docs ou outros formatos.*
        """
        return documento

class AgenteFinal:
    def apresentar_resultado(self, resultado):
        """Apresenta o resultado final para o usuário (agora retorna o texto)."""
        return resultado if resultado is not None else "" # Garante que um string seja retornado

# --- Função principal para ser chamada pelo frontend ---
def iniciar_processo(opcao, tema):
    resultado_agente = None

    if opcao == 1: # Pesquisa com resumo em tópicos
        agente_pesquisador = AgentePesquisador()
        resultado_agente = agente_pesquisador.processar(tema)

    elif opcao == 2: # Questionário com gabarito
        agente_questionador = AgenteQuestionador()
        resultado_agente = agente_questionador.processar(tema)

    elif opcao == 3: # Cronograma de estudos
        agente_planejador = AgentePlanejador()
        resultado_agente = agente_planejador.processar(tema)

    elif opcao == 4: # Completão
        agente_pesquisador = AgentePesquisador()
        pesquisa = agente_pesquisador.processar(tema)

        agente_questionador = AgenteQuestionador()
        questionario = agente_questionador.processar(tema)

        agente_planejador = AgentePlanejador()
        cronograma = agente_planejador.processar(tema)

        agente_professor = AgenteProfessor()
        resultado_agente = agente_professor.compilar_info(pesquisa, questionario, cronograma)

    if resultado_agente is not None:
        agente_final = AgenteFinal()
        return agente_final.apresentar_resultado(resultado_agente)
    else:
        return "Nenhuma opção válida foi selecionada."

**Frontend**

In [35]:
import ipywidgets as widgets
from IPython.display import display
from IPython.display import Markdown

# Criação dos widgets com tooltips
label_instrucao = widgets.Label(value="Escolha uma das opções:")
botao_pesquisa = widgets.Button(description="Pesquisar",
                                tooltip="Gera um resumo em tópicos sobre o tema.",
                                style=widgets.ButtonStyle())  # Estilo inicial padrão
botao_questionario = widgets.Button(description="Quiz",
                                    tooltip="Cria um questionário com gabarito sobre o tema.",
                                    style=widgets.ButtonStyle())
botao_cronograma = widgets.Button(description="Cronograma",
                                    tooltip="Elabora um cronograma de estudos para o tema.",
                                    style=widgets.ButtonStyle())
botao_completao = widgets.Button(description="Completão",
                                  tooltip="Gera pesquisa, questionário e cronograma completos.",
                                  style=widgets.ButtonStyle())
campo_tema = widgets.Text(description="Tema:")
botao_enviar = widgets.Button(description="Gerar!")
area_resultado = widgets.Output() # Usaremos Output para exibir Markdown
rodape_info = widgets.HTML(value='<div style="position: fixed; bottom: 10px; left: 10px; font-size: small;">'
                                 'Assistente de Estudos Versão ALPHA</div>'
                                 '<div style="position: fixed; bottom: 10px; right: 10px; font-size: small; text-align: right;">'
                                 f'Instagram: <a href="https://www.instagram.com/prof.jvictor/" target="_blank">@prof.jvictor</a><br>'
                                 f'LinkedIn: <a href="https://www.linkedin.com/in/jvictorLL/" target="_blank">jvictorLL</a></div>')
logo_alura = widgets.HTML(value='<div style="position: absolute; top: 0px; right: 10px; width: auto; overflow: hidden;">'
                               '<img src="https://cursos.alura.com.br/assets/images/immersion/imersao_ia_google_logo.png" height="50"></div>')

# Organização da interface principal com maior margem inferior para a área de resultado
interface_principal = widgets.VBox([
    widgets.HBox([label_instrucao, logo_alura], layout=widgets.Layout(justify_content='space-between', min_height='60px')),
    widgets.HBox([botao_pesquisa, botao_questionario, botao_cronograma, botao_completao]),
    campo_tema,
    botao_enviar,
    widgets.VBox([area_resultado], layout=widgets.Layout(margin='0 0 80px 0')) # Aumentei a margem inferior para 80px
])

# Estilo para o botão ativo
botao_ativo_style = widgets.ButtonStyle(button_color='lightblue')
botao_padrao_style = widgets.ButtonStyle()

# Variável para controlar o botão ativo
botao_ativo = None
opcao_selecionada = None

def resetar_estilo_botoes():
    """Reseta o estilo de todos os botões para o padrão."""
    botao_pesquisa.style = botao_padrao_style
    botao_questionario.style = botao_padrao_style
    botao_cronograma.style = botao_padrao_style
    botao_completao.style = botao_padrao_style

def on_botao_pesquisa_clicked(b):
    global opcao_selecionada, botao_ativo
    opcao_selecionada = 1
    resetar_estilo_botoes()
    botao_pesquisa.style = botao_ativo_style
    botao_ativo = botao_pesquisa

def on_botao_questionario_clicked(b):
    global opcao_selecionada, botao_ativo
    opcao_selecionada = 2
    resetar_estilo_botoes()
    botao_questionario.style = botao_ativo_style
    botao_ativo = botao_questionario

def on_botao_cronograma_clicked(b):
    global opcao_selecionada, botao_ativo
    opcao_selecionada = 3
    resetar_estilo_botoes()
    botao_cronograma.style = botao_ativo_style
    botao_ativo = botao_cronograma

def on_botao_completao_clicked(b):
    global opcao_selecionada, botao_ativo
    opcao_selecionada = 4
    resetar_estilo_botoes()
    botao_completao.style = botao_ativo_style
    botao_ativo = botao_completao

def on_botao_enviar_clicked(b):
    global opcao_selecionada
    with area_resultado:
        area_resultado.clear_output()
        if opcao_selecionada is not None and campo_tema.value:
            resultado = iniciar_processo(opcao_selecionada, campo_tema.value)
            display(Markdown(resultado)) # Exibe o resultado formatado
        elif not campo_tema.value:
            display(widgets.Label(value="Por favor, digite um tema de estudo."))
        else:
            display(widgets.Label(value="Por favor, selecione uma opção."))

# Associa as funções aos eventos de clique dos botões
botao_pesquisa.on_click(on_botao_pesquisa_clicked)
botao_questionario.on_click(on_botao_questionario_clicked)
botao_cronograma.on_click(on_botao_cronograma_clicked)
botao_completao.on_click(on_botao_completao_clicked)
botao_enviar.on_click(on_botao_enviar_clicked)

# Exibe a interface principal e o rodapé separadamente
display(interface_principal)
display(rodape_info)

HTML(value='<div style="position: fixed; bottom: 10px; left: 10px; font-size: small;">Assistente de Estudos Ve…